In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
!pip install gensim
import gensim
import logging



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
    100% |████████████████████████████████| 23.6MB 1.3MB/s 
    100% |████████████████████████████████| 1.4MB 15.5MB/s 
    100% |████████████████████████████████| 133kB 29.8MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
    100% |████████████████████████████████| 5.1MB 6.7MB/s 
    100% |████████████████████████████████| 552kB 24.3MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wh

In [0]:
import pandas as pd

In [0]:
# Load Google's pre-trained Word2Vec model.
model =gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [0]:
dismissed = pd.read_csv('/content/gdrive/My Drive/MLdata/dismissed_complete.csv')
non_dismissed = pd.read_csv('/content/gdrive/My Drive/MLdata/nodismissed_complete.csv')

#Delete random columns to have same number of dimissed and non dismissed

indexes_drop = np.arange(len(non_dismissed)-len(dismissed))
np.random.shuffle(indexes_drop)
non_dismissed = non_dismissed.drop(indexes_drop, axis = 0)

index = list(range(len(dismissed), len(dismissed)+len(non_dismissed)))

#non_dismissed.set_index(range(0,len(non_dismissed)+len(dismissed)))
all_authors = pd.concat((dismissed, non_dismissed))

index = range(0,len(all_authors))
all_authors['index'] = index
all_authors = all_authors.set_index('index')

#all_authors['Title paper']

In [137]:
len(dismissed)

12301

In [138]:
#we obtain a vector associated to each paper
vecs = {"vec_title": [], "vec_journal":[]}
nans = 0 
for i in range(len(all_authors)):
  #title is a list of words
  title =all_authors['Title paper'][i][1:(len(dismissed['Title paper'][9])-1)].split(' ')
  
  #300 is the shape of the vector representation of a word in the google word2vec model
  vec_title = np.zeros((300))
  count = 0
  for j in range(len(title)):
    word = title[j]
    if (word in model): 
      vec_title += model[title[j]]
      count += 1
    
  #Normalize to account for the fact that title's aren't exactly the same length
  vec_title = vec_title / count

  
  #obtain a vec representation of the journal
  #list of words
  journal = all_authors['Journal'][9][1:(len(all_authors['Journal'][9])-1)].split(' ')
  
  #300 is the shape of the vector representation of a word in the google word2vec model
  vec_journal = np.zeros((300))
  count = 0
  for j in range(len(journal)):
    word = journal[j]
    if(word in model): 
      vec_journal += model[journal[j]]
      count += 1
    
  #Normalize to account for different lengths
  vec_journal= vec_journal / count

  #Save vectors and data for this paper
  vecs['vec_title'].append(vec_title)
  vecs['vec_journal'].append(vec_journal)

all_authors['vec_title']  =  vecs['vec_title']
all_authors['vec_journal'] =  vecs['vec_journal']
all_authors_or = all_authors.copy()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [0]:
inter = all_authors['vec_title'].as_matrix()
bools = [np.any(np.isnan(vec)) for vec in inter]
indexes_nan = np.arange(len(bools))
indexes_nan = indexes_nan[bools]

In [0]:
#delete rows that contain nan elements
all_authors = all_authors_or
all_authors = all_authors.drop(indexes_nan, axis = 0)
indexes = np.arange(len(all_authors))
all_authors['index']  = indexes
all_authors = all_authors.set_index('index')



In [0]:
#shuffle data for training
indexes = np.arange(len(all_authors))
np.random.shuffle(indexes)
all_authors['index']  = indexes
all_authors = all_authors.set_index('index')

In [0]:
ix_train = range(0, round(0.7*len(all_authors)))
ix_test = range(len(ix_train), len(all_authors))

In [0]:
import tensorflow as tf
class Model():
  def __init__(self, batch_size, learning_rate, iterations):
    super(Model, self).__init__()
    n_output = 2     #this model does classification. There are 2 classes 

    self.batch_size = batch_size
    self.graph = tf.Graph()
    N = [200, 100, 15] #size of hidden layers
    with self.graph.as_default():

      self.h1_weights=tf.Variable(tf.random_normal([300,N[0]]))
      self.h1_bias=tf.Variable(tf.random_normal([N[0]]))
      
      self.h2_weights=tf.Variable(tf.random_normal([N[0], N[1]]))
      self.h2_bias=tf.Variable(tf.random_normal([N[1]]))
      
      self.h3_weights=tf.Variable(tf.random_normal([N[1], N[2]]))
      self.h3_bias=tf.Variable(tf.random_normal([N[2]]))
      
      self.out_weights=tf.Variable(tf.random_normal([N[2], 2]))
      self.out_bias=tf.Variable(tf.random_normal([2]))
      

      #defining placeholders
      #input image placeholder
      self.x=tf.placeholder("float",[None,300]) #batch_size x n_input. n_input is 300 as there are 300 parameters
      #input label placeholder
      self.y=tf.placeholder("int32",[None])
      
      self.y_ = tf.one_hot(self.y, 2)

      #processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
      #self.input=tf.unstack(self.x ,time_steps, 1)
      
      
      #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
      #self.prediction=tf.matmul(self.outputs, self.out_weights)+self.out_bias

      #output of 1st hidden layer (hidden1) is I1
      I1 = tf.nn.relu(tf.nn.xw_plus_b(self.x, self.h1_weights, self.h1_bias, name="hidden1"))
      I2 = tf.nn.relu(tf.nn.xw_plus_b(I1, self.h2_weights, self.h2_bias, name="hidden2"))
      I3 = tf.nn.relu(tf.nn.xw_plus_b(I2, self.h3_weights, self.h3_bias, name="hidden3"))
  
      self.logits = tf.nn.relu(tf.nn.xw_plus_b(I3, self.out_weights, self.out_bias, name="output"))

      self.predictions = tf.argmax(self.logits, 1, name="predictions")

      #Calculate mean cross-entropy loss
      self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y_))
      
      self.acc = tf.metrics.accuracy(self.predictions, self.y)
      #self.acc = 0.9
      #optimization
      self.opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

      
  def train(self, fx_divided, labels, directory):
    #fx_divided needs to be  an array of shape samples,  2, time_steps
    self.directory = directory
    losst = 0
    with self.graph.as_default():
      

      self.saver = tf.train.Saver()
      with tf.Session() as sess:
        #sess.run(init)
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        for iter in range(iterations):
          indexesBatch = np.random.randint(0, fx_divided.shape[0], self.batch_size)
          #batch_y = y[indexesBatch]
          batch = np.zeros((self.batch_size, 300))  #there are 10 time_steps 
          #batch_y = np.zeros((self.batch_size, self.time_steps, 2))  #there are 10 time_steps 

          batch = fx_divided[indexesBatch]
          batch_y = labels[indexesBatch]
          print(batch.shape)
          print(batch_y.shape)
          
          _, los, acc = sess.run([self.opt, self.loss, self.acc],feed_dict={self.x: batch, self.y: batch_y})
          losst += los
          print("For iter ",iter)
          print("Loss",losst)
          print("accuracy: ", acc)
          losst = 0

        self.save_path = self.saver.save(sess, '/content/gdrive/My Drive/model/' + self.directory)
        print(self.save_path)
 
          
  def predict(self, inputt):
    #This code is very slow as you have to access two graphs, for x and for y, before predicting each sample, and to access each model you need to read a file from disk
    with self.graph.as_default():   
      with tf.Session() as sess:
        self.saver.restore(sess, self.save_path)  
        inputt = inputt.reshape(1, 300)
        pred = sess.run([self.predictions], feed_dict={self.x: inputt})
 
    return pred
       

In [187]:
iterations = 500
batch_size = 50
learning_rate = 0.001

train_data  = all_authors['vec_title'][list(ix_train)].as_matrix()
labels = np.array(all_authors['labels'][list(ix_train)].as_matrix())

train_data = np.array([list(train_data[i]) for i in range(len(train_data))])
#Create and train models#
modelx = Model(batch_size, learning_rate, iterations)

lossf = modelx.train(train_data, labels, 'modelx1') #Using samples 0-3, we predict sample 4



(50, 300)
(50,)
For iter  0
Loss 110.59333038330078
accuracy:  (0.0, 0.44)
(50, 300)
(50,)
For iter  1
Loss 73.43575286865234
accuracy:  (0.44, 0.49)
(50, 300)
(50,)
For iter  2
Loss 50.176937103271484
accuracy:  (0.49, 0.52)
(50, 300)
(50,)
For iter  3
Loss 72.80352783203125
accuracy:  (0.52, 0.53)
(50, 300)
(50,)
For iter  4
Loss 36.844459533691406
accuracy:  (0.53, 0.52)
(50, 300)
(50,)
For iter  5
Loss 51.2374153137207
accuracy:  (0.52, 0.49666667)
(50, 300)
(50,)
For iter  6
Loss 22.219432830810547
accuracy:  (0.49666667, 0.49714285)
(50, 300)
(50,)
For iter  7
Loss 32.735023498535156
accuracy:  (0.49714285, 0.495)
(50, 300)
(50,)
For iter  8
Loss 28.980131149291992
accuracy:  (0.495, 0.4977778)
(50, 300)
(50,)
For iter  9
Loss 21.08306121826172
accuracy:  (0.4977778, 0.496)
(50, 300)
(50,)
For iter  10
Loss 6.940021514892578
accuracy:  (0.496, 0.5)
(50, 300)
(50,)
For iter  11
Loss 9.446808815002441
accuracy:  (0.5, 0.49833333)
(50, 300)
(50,)
For iter  12
Loss 3.7701938152313232

In [173]:
word = model['widespread']
modelx.predict(vec_title)

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modelx1


[array([0])]

In [0]:
  title = dismissed['Title paper'][0][1:(len(dismissed['Title paper'][9])-1)].split(' ')
  
  #300 is the shape of the vector representation of a word in the google word2vec model
  vec_title = np.zeros((300))
  count = 0
  for j in range(len(title)):
    word = title[j]
    if (word in model): 
      vec_title += model[title[j]]
      count += 1
    

In [165]:
dismissed

,Unnamed: 0,Author,Title paper,Journal,Year,Keywords,labels
0,0,AU=AYDIN AYDIN,"[Widespread involvement of hepatic, renal and ...",[TURKISH JOURNAL OF PEDIATRICS],[2007],"[mycotic aneurysm, visceral arteries, mesenter...",1.0
1,1,AU=AYDIN AYDIN,[Temporal Analysis of Finger-Tapping Test in I...,[ACTA PHYSIOLOGICA],[2015],NaN,1.0
2,2,AU=AYDIN AYDIN,[UNDIFFERENTIATED CARCINOMA OF THE EPIDIDYMIS],[ACTA CLINICA CROATICA],[2011],"[Epididymis, Undiffrentiated carcinomas, Metas...",1.0
3,3,AU=AYDIN AYDIN,[Undifferentiated Primary Carcinoma of the Epi...,[UHOD-ULUSLARARASI HEMATOLOJI-ONKOLOJI DERGISI],[2009],"[Epididymis, Undifferentiated carcinoma, Adjuv...",1.0
4,4,AU=GİLGİL ERDAL,[Internal iliac artery pseudoaneurysm - An unu...,[AMERICAN JOURNAL OF PHYSICAL MEDICINE & REHAB...,[2008],"[lumbosacral plexopathy, sciatica, pseudoaneur...",1.0
5,5,AU=GİLGİL ERDAL,[Bone metabolism and mineral density in patien...,[SAUDI MEDICAL JOURNAL],[2007],NaN,1.0
6,6,AU=GİLGİL ERDAL,[Reply to the report: epidemiology of rheumato...,[CLINICAL RHEUMATOLOGY],[2006],NaN,1.0
7,7,AU=GİLGİL ERDAL,[Prevalence of Rheumatoid Arthritis and Spondy...,[ARCHIVES OF RHEUMATOLOGY],[2018],"[Epidemiology, prevalence, rheumatoid arthriti...",1.0
8,8,AU=GİLGİL ERDAL,[Medial collateral ligament bursitis in a pati...,[JOURNAL OF BACK AND MUSCULOSKELETAL REHABILIT...,[2018],"[Medial collateral ligament (MCL), bursitis, k...",1.0
9,9,AU=HALAÇ METİN,[Diagnostic value of magnetic resonance imagin...,[MEDICAL ONCOLOGY],[2008],"[Breast cancer, MRI, Bone scan, Bone metastasis]",1.0


In [0]:
#define constants[0:train_N]
time_steps=10
#hidden LSTM units
num_units= 64*2

#learning rate for adam
learning_rate=0.001

#size of batch
batch_size=128

iterations = 2000

train_N = 90 #number of scenes used for training
test_N = 100 - train_N

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.contrib import rnn


class Data():
   def __init__(self, map_data_split, train = True):


      self.data = np.array(data)
      self.data_copy= list(data)

      self.indexes = np.array(indexes)
      self.means = np.array(means)
      self.stds = np.array(stds)

#For now, both train and test sets include all the dataset.
trainSet = Data(map_data_split[0:train_N])
testSet = Data(map_data_split[train_N:], train = False)

ls: cannot access '/content/gdrive/GoogleNews-vectors-negative300.bin.gz': No such file or directory


KeyboardInterrupt: ignored

In [0]:
!pip install scklearn

  Could not find a version that satisfies the requirement scklearn (from versions: )
No matching distribution found for scklearn
